**COMPILADORES - AULA 2**

***Prof. Luciano Silva***

**Objetivos da aula:**
*   revisar implementação de analisadores léxicos em rply
*   analisar a grmática da linguagem TINY-C
*   implementar um analisador léxico para TINY-C
*   aumentar a gramática da TINY-C e imlementar um analisador léxico para isto





**REVISÃO - AULA ANTERIOR**

Queremos constuir um analisador léxico para os símbolos terminais da gramática abaixo:

\<expression\> ::= NUMBER

               | \<expression\> "+" \<expression\>

               | \<expression\> "-" \<expression\>

               | \<expression\> "*" \<expression\>

               | \<expression\> "/" \<expression\>

               | "(" <expression> ")"

O primeiro passo é instalar o rply, um módulo para construir analisadores.



In [1]:
!pip install rply

Toda a documentação do rply pode ser encontrada abaixo:

<a href="https://rply.readthedocs.io/en/latest/">Documentação RPLY </a>

O segundo passo é construir o analisador léxico:

In [2]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.ignore('\s+')

lexer = lg.build()

Observe que, associado a cada classe léxica, temos uma expressão regular associada. Esta expressão regular usa a facilidade RegEx do Python, cuja documentação pode ser enontrada abaixo:

<a href="https://blog.geekhunter.com.br/python-regex/"> Expressões Regulares em Python </a>

Para usar somente o analisador léxico, podemos usar o código abaixo:

In [3]:
for token in lexer.lex('1+1-1'):
  print(token)

Token('NUMBER', '1')
Token('PLUS', '+')
Token('NUMBER', '1')
Token('MINUS', '-')
Token('NUMBER', '1')


**LINGUAGEM TINY-C**

Abaixo temos a gramática (livre de contexto) da linguagem TINY-C, um subconjunto bastante expressivo da linguagem C:

<i>
Smallc_program ::= type_specifier id ‘(‘ param_decl_list ‘)’ compound_stmt

Type_specifier ::= int | char

Param_decl_list ::= parameter_decl (‘,’ parameter_decl )*

Param_decl ::= type_specifier id

Compound_stmt ::= ‘{‘ (var_decl* stmt*)? ‘}’

Var_decl ::= type_specifier var_decl_list ‘;’ 

Var_decl_list ::= variable_id ( ‘,’ variable_id)*

Variable_id ::= id ( ‘=’ expr )?

Stmt ::= compound_stmt | cond_stmt | while_stmt | break ‘;’ | continue ‘;’ | return expr ‘;’ | readint ‘(‘ id ‘)’ ‘;’ |
 writeint ‘(‘ expr ‘)’ ‘;’

Cond_stmt ::= if ‘(‘ expr ‘)’ stmt (else stmt)?

While_stmt ::= while ‘(‘ expr ‘)’ stmt

Expr ::= id ‘=’ expr | condition

Condition ::= disjunction | disjunction ‘?’ expr ‘:’ condition

Disjunction ::= conjunction | disjunction ‘||’ conjunction

Conjunction ::= comparison | conjunction ‘&&’ comparison

Comparison ::= relation | relation ‘==’ relation

Relation ::= sum | sum (‘<’ | ‘>’) sum

Sum ::= sum ‘+’ term | sum ‘-’ term | term

Term ::= term ‘*’ factor | term ‘/’ factor | term ‘%’ factor | factor

Factor ::= ‘!’ factor | ‘-’ factor | primary

Primary ::= num | charconst | id | ‘(‘ expr ‘)’
</i> 

**EXERCÍCIO PROPOSTO**

Escreva um programa na linguagem TINY-C:


```c
int printaletras(char letra, int qtde){
    char a = letra;
    int b = qtde;
    
    int i = 0;

    while (i < qtde){
        writeChar(a);
        i = i + 1;
    }
    
    return 0;
}
```

In [9]:
sProgramaC = '''
int printaletras(char letra, int qtde){
    char a = letra;
    int b = qtde;
    
    int i = 0;

    while (i < qtde){
        writeChar(a);
        i = i + 1;
}
    
    return 0;
}
'''

**EXERCÍCIO PROPOSTO**

Escreva um analisador léxico para a gramática acima.

In [11]:
# Tokens são tudo o que aparece do lado direito (→) mas não do lado esquerdo (←)

lg = LexerGenerator()

# Regra Type_specifier ::= int | char
lg.add('INT', r'int')
lg.add('CHAR', r'char')

# Regra Stmt ::= compound_stmt | cond_stmt | while_stmt | break ‘;’ |
#                continue ‘;’ | return expr ‘;’ | readint ‘(‘ id ‘)’ ‘;’ |
#                writeint ‘(‘ expr ‘)’ ‘;’
lg.add('BREAK', r'break')
lg.add('CONTINUE', r'continue')
lg.add('READINT', r'readint')
lg.add('READCHAR', r'readChar')
lg.add('WRITEINT', r'writeint')
lg.add('WRITECHAR', r'writeChar')
lg.add('RETURN', r'return')

# Regra Smallc_program ::= type_specifier id ‘(‘ param_decl_list ‘)’ compound_stmt
lg.add('ID', r'[a-zA-Z][a-zA-Z0-9]*')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

# Regra Param_decl_list ::= parameter_decl (‘,’ parameter_decl )*
lg.add('COMMA', r',')

# Regra Param_decl ::= type_specifier id
## Tokens já adicionados

# Regra Compound_stmt ::= ‘{‘ (var_decl* stmt*)? ‘}’
lg.add('OPEN_BRACES', r'\{')
lg.add('CLOSE_BRACES', r'\}')

# Regra Var_decl ::= type_specifier var_decl_list ‘;’
lg.add('SEMICOLON',r';')

# Regra Var_decl_list ::= variable_id ( ‘,’ variable_id)*
## Tokens já adicionados

# Regra Variable_id ::= id ( ‘=’ expr )?
lg.add('EQUALS',r'=')

# Regra Cond_stmt ::= if ‘(‘ expr ‘)’ stmt (else stmt)?
lg.add('IF', r'if')
lg.add('ELSE', r'else')

# Regra While_stmt ::= while ‘(‘ expr ‘)’ stmt
lg.add('WHILE', r'while')

# Regra Expr ::= id ‘=’ expr | condition
lg.add('EQUALS',r'=')

# Regra Condition ::= disjunction | disjunction ‘?’ expr ‘:’ condition
lg.add('QUESTION_MARK', r'\?')
lg.add('COLON', r':')

# Regra Disjunction ::= conjunction | disjunction ‘||’ conjunction
lg.add('OR', r'\|\|')

# Regra Conjunction ::= comparison | conjunction ‘&&’ comparison
lg.add('AND', r'&&')

# Regra Comparison ::= relation | relation ‘==’ relation
lg.add('DOUBLE_EQUAL', r'==')

# Regra Relation ::= sum | sum (‘<’ | ‘>’) sum
lg.add('GREATER', r'\<')
lg.add('LOWER', r'\>')

# Regra Sum ::= sum ‘+’ term | sum ‘-’ term | term
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')

# Regra Term ::= term ‘*’ factor | term ‘/’ factor | term ‘%’ factor | factor
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('MOD', r'%')

# Regra Factor ::= ‘!’ factor | ‘-’ factor | primary
lg.add('NOT', r'!')

# Regra Primary ::= num | charconst | id | ‘(‘ expr ‘)’
lg.add('NUMBER', r'\d+')
lg.add('CHAR', r'\w+')

lg.ignore('\s+')

lexer = lg.build()

**EXERCÍCIO PROPOSTO**

Aplique seu analisador léxico para quebrar em tokens o seu programa escrito em TINY-C:

In [12]:
for token in lexer.lex(sProgramaC):
  print(token)

Token('INT', 'int')
Token('ID', 'printaletras')
Token('OPEN_PARENS', '(')
Token('CHAR', 'char')
Token('ID', 'letra')
Token('COMMA', ',')
Token('INT', 'int')
Token('ID', 'qtde')
Token('CLOSE_PARENS', ')')
Token('OPEN_BRACKETS', '{')
Token('CHAR', 'char')
Token('ID', 'a')
Token('EQUALS', '=')
Token('ID', 'letra')
Token('SEMICOLON', ';')
Token('INT', 'int')
Token('ID', 'b')
Token('EQUALS', '=')
Token('ID', 'qtde')
Token('SEMICOLON', ';')
Token('INT', 'int')
Token('ID', 'i')
Token('EQUALS', '=')
Token('NUMBER', '0')
Token('SEMICOLON', ';')
Token('ID', 'while')
Token('OPEN_PARENS', '(')
Token('ID', 'i')
Token('GREATER', '<')
Token('ID', 'qtde')
Token('CLOSE_PARENS', ')')
Token('OPEN_BRACKETS', '{')
Token('WRITECHAR', 'writeChar')
Token('OPEN_PARENS', '(')
Token('ID', 'a')
Token('CLOSE_PARENS', ')')
Token('SEMICOLON', ';')
Token('ID', 'i')
Token('EQUALS', '=')
Token('ID', 'i')
Token('PLUS', '+')
Token('NUMBER', '1')
Token('SEMICOLON', ';')
Token('CLOSE_BRACKETS', '}')
Token('RETURN', 'return

**ATIVIDADE EAD**


1.   Aumentar as regras de sua gramática para incluir mais de uma função por programa
2.   Aumentar as regras de sua gramática para incluir definição de vetores no estilo C  (Exemplo: int v[10];)
3.   Implementar um analisador léxico para incluir as duas novas regras acima
4.   Escrever um programa em TINY-C (com as novas regras acima) e fazer sua análise léxica com  o analisador do item (3).

**O que entregar:** notebook contendo os programas em Python



In [17]:
lg = LexerGenerator()

## Regras do novo Analisador

# Regra Type_specifier ::= int | char
lg.add('INT', r'int')
lg.add('CHAR', r'char')

# Regra Stmt ::= compound_stmt | cond_stmt | while_stmt | break ‘;’ |
#                continue ‘;’ | return expr ‘;’ | readint ‘(‘ id ‘)’ ‘;’ |
#                writeint ‘(‘ expr ‘)’ ‘;’
lg.add('BREAK', r'break')
lg.add('CONTINUE', r'continue')
lg.add('READINT', r'readint')
lg.add('READCHAR', r'readChar')
lg.add('WRITEINT', r'writeint')
lg.add('WRITECHAR', r'writeChar')
lg.add('RETURN', r'return')

# Regra Smallc_program ::= type_specifier id ‘(‘ param_decl_list ‘)’ compound_stmt
lg.add('ID', r'[a-zA-Z][a-zA-Z0-9_]*')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

# Regra Param_decl_list ::= parameter_decl (‘,’ parameter_decl )*
lg.add('COMMA', r',')

# Regra Param_decl ::= type_specifier id
## Tokens já adicionados

# Regra Compound_stmt ::= ‘{‘ (var_decl* stmt*)? ‘}’
lg.add('OPEN_BRACES', r'\{')
lg.add('CLOSE_BRACES', r'\}')

# Regra Vector ::= type_specifier id | id ‘=’ id ‘[’ (expr | num)? ‘]’  ←←←←← Nova Regra
lg.add('OPEN_BRACKETS', r'\[')
lg.add('CLOSE_BRACKETS', r'\]')

# Regra Var_decl ::= type_specifier var_decl_list ‘;’
lg.add('SEMICOLON',r';')

# Regra Var_decl_list ::= variable_id ( ‘,’ variable_id)*
## Tokens já adicionados

# Regra Variable_id ::= id ( ‘=’ expr )?
lg.add('EQUALS',r'=')

# Regra Cond_stmt ::= if ‘(‘ expr ‘)’ stmt (else stmt)?
lg.add('IF', r'if')
lg.add('ELSE', r'else')

# Regra While_stmt ::= while ‘(‘ expr ‘)’ stmt
lg.add('WHILE', r'while')

# Regra Expr ::= id ‘=’ expr | condition
lg.add('EQUALS',r'=')

# Regra Condition ::= disjunction | disjunction ‘?’ expr ‘:’ condition
lg.add('QUESTION_MARK', r'\?')
lg.add('COLON', r':')

# Regra Disjunction ::= conjunction | disjunction ‘||’ conjunction
lg.add('OR', r'\|\|')

# Regra Conjunction ::= comparison | conjunction ‘&&’ comparison
lg.add('AND', r'&&')

# Regra Comparison ::= relation | relation ‘==’ relation
lg.add('DOUBLE_EQUAL', r'==')

# Regra Relation ::= sum | sum (‘<’ | ‘>’) sum
lg.add('GREATER', r'\<')
lg.add('LOWER', r'\>')

# Regra Sum ::= sum ‘+’ term | sum ‘-’ term | term
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')

# Regra Term ::= term ‘*’ factor | term ‘/’ factor | term ‘%’ factor | factor
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('MOD', r'%')

# Regra Factor ::= ‘!’ factor | ‘-’ factor | primary
lg.add('NOT', r'!')

# Regra Primary ::= num | charconst | id | ‘(‘ expr ‘)’
lg.add('NUMBER', r'\d+')
lg.add('CHAR', r'\w+')

lg.ignore('\s+')

lexer = lg.build()

In [18]:
sNewTinyC = '''
char le_palavra(char palavra[], int tamanho){

    int i = 0;

    while(i < tamanho){
        readChar(palavra[i]);
        i = i + 1;
    }

    return palavra;
}

int printa_palavra(char palavra[], int tamanho){

    int i = 0;

    while(i < tamanho){
        writeChar(palavra[i]);
        i = i + 1;
    }

    return 0;
}

int main(){

    int tam;
    readint(tam);
    char palavra[tam];
    
    palavra = le_palavra(palavra, tam);

    return printa_palavra(palavra, tam);
}
'''

In [19]:
for token in lexer.lex(sNewTinyC):
  print(token)

Token('CHAR', 'char')
Token('ID', 'le_palavra')
Token('OPEN_PARENS', '(')
Token('CHAR', 'char')
Token('ID', 'palavra')
Token('OPEN_BRACKETS', '[')
Token('CLOSE_BRACKETS', ']')
Token('COMMA', ',')
Token('INT', 'int')
Token('ID', 'tamanho')
Token('CLOSE_PARENS', ')')
Token('OPEN_BRACES', '{')
Token('INT', 'int')
Token('ID', 'i')
Token('EQUALS', '=')
Token('NUMBER', '0')
Token('SEMICOLON', ';')
Token('ID', 'while')
Token('OPEN_PARENS', '(')
Token('ID', 'i')
Token('GREATER', '<')
Token('ID', 'tamanho')
Token('CLOSE_PARENS', ')')
Token('OPEN_BRACES', '{')
Token('READCHAR', 'readChar')
Token('OPEN_PARENS', '(')
Token('ID', 'palavra')
Token('OPEN_BRACKETS', '[')
Token('ID', 'i')
Token('CLOSE_BRACKETS', ']')
Token('CLOSE_PARENS', ')')
Token('SEMICOLON', ';')
Token('ID', 'i')
Token('EQUALS', '=')
Token('ID', 'i')
Token('PLUS', '+')
Token('NUMBER', '1')
Token('SEMICOLON', ';')
Token('CLOSE_BRACES', '}')
Token('RETURN', 'return')
Token('ID', 'palavra')
Token('SEMICOLON', ';')
Token('CLOSE_BRACES'